In [21]:
import math

# Numpy is only required to save the key maps as a nd array and for the test routines
import numpy as np

In [22]:
# Capturing all the possible maps from each key in the dialpad as 1 for all the possible edges in the graph
# where each key is represented as a node respectively.

possible_dials = {
  0: (0, 0, 0, 0, 1, 0, 1, 0, 0, 0),
  1: (0, 0, 0, 0, 0, 0, 1, 0, 1, 0),
  2: (0, 0, 0, 0, 0, 0, 0, 1, 0, 1),
  3: (0, 0, 0, 0, 1, 0, 0, 0, 1, 0),
  4: (1, 0, 0, 1, 0, 0, 0, 0, 0, 1),
  5: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
  6: (1, 1, 0, 0, 0, 0, 0, 1, 0, 0),
  7: (0, 0, 1, 0, 0, 0, 1, 0, 0, 0),
  8: (0, 1, 0, 1, 0, 0, 0, 0, 0 ,0),
  9: (0, 0, 1, 0, 1, 0, 0, 0, 0, 0),
}

In [23]:
# Converting the map above into a matrix for further operations.

matrix_possible_dials = np.float64(np.vstack((possible_dials[0],possible_dials[1],possible_dials[2],
                                   possible_dials[3],possible_dials[4],possible_dials[5],
                                   possible_dials[6],possible_dials[7],possible_dials[8],
                                   possible_dials[9])))

## The count for 0 hops is trivial as there is only 1 possibility for each key. But for number of hops, N > 1, the count of possibilities for each key is represented by the elements of the vector obtained by N times matrix vector product starting from the trivial case with vector of all entries as 1's (i.e N = 0), and replacing the new vector every time starting from N = 0. Each element in the final vector represents the number of possibilities for each key starting from 0 to 9.

## The shape of the Matrix is (10,10) and the vector is (10,1). But the sparsity of the matrix would allow us to further exploit the problem and store the Matrix in the compressed row storage (CRS) format. For large N's, it would make an impact as the matrix has only 20 non zero elements and requires only 20 element wise product to compute a new vector per each hop as opposed to 1000 for the dense matrix with zeros.

## CRS or Yale format consist of three arrays 1) v - non zero values of A. 2) j - column indices of the values. 3) i - to store the
## beginning and end of each row in form of the first index of the current and the next row. The lenght of i is always M + 1 if M is the number of rows

# Solving the same problem Without using built in modules

In [24]:
# Function to convert a given dense matrix into a sparse Compressed row storage format.

def crs_storage(dense_matrix):
    non_zero_values = []
    column_index = []
    row_count = [0]    # Initialize the count of non-zero elements before the beginning of the first row to 0
    counter = 0
    for i in range(dense_matrix.shape[0]):      # Tracing the rows
        for j in range(dense_matrix.shape[1]):  # Tracing the columns
            if(dense_matrix[i][j] != 0):
                non_zero_values.append(dense_matrix[i][j])
                column_index.append(j)
                counter = counter + 1
        row_count.append(counter)
    
    return (non_zero_values, column_index, row_count)

In [25]:
# CRS Matrix Vector multiplication

def crs_matrix_vector_multiplication(values, column, row, vector_count, vector_count_new):
    for i in range(len(row)-1):
        for j in range(row[i], row[i+1], 1):
            vector_count_new[i] += (vector_count[column[j]]*values[j])
    return vector_count_new

In [26]:
# Initiate count arrays for each key in the dialpad by 1 as we can press each key for the hops=0

def initiate_count_array(count_array_temp):
    for k in range(matrix_possible_dials.shape[0]):
        count_array_temp.append(1)
    return count_array_temp

In [27]:

# Function to calculate the sum of all the elements of a vector

def arraySum(array_temp):
    result = 0
    for i in range(len(array_temp)):
        result += array_temp[i]
    return result

In [28]:
# Function to calculate the euclidean norm of a vector

def vector_euclidean_norm(vector_x):
    result_norm = 0.0
    for elem in range(len(vector_x)):
        result_norm += (vector_x[elem] * vector_x[elem])
    return math.sqrt(result_norm)

In [29]:
# Function to compute total number of possible dials starting fram each key based on given "N" hops

def count_N_runs(N):
    count_array_result = []
    count_array_result = initiate_count_array(count_array_result)
    for i in range(N):
        Count_array_temp = [0] * matrix_possible_dials.shape[0]
        count_array_result = crs_matrix_vector_multiplication(NonZeroValues, ColumnIndex, RowCount, 
                             count_array_result, Count_array_temp)
    return count_array_result

# Unit tests to check the correctness of each modular functions independent of the number of hops

In [30]:
from scipy.sparse import csr_matrix, random

# Test-1 tests the correctness of "CRS_Storage", "CRSMatrix_vector_Multiplication" and "vector_eucledian_norm" functions given a 
# dense matrix and a vector of matching shapes as input for test routine
def test_1(dense_matrix, array_input, array_rhs, test_failure_count, epsilon):
    assertion_statement = "Mismatch in dimension and matrix vector product not possible"
    assert dense_matrix.shape[1] == len(array_input), assertion_statement

    # Converting any given dense matrix into CRS sparse representation
    values, column, row = crs_storage(dense_matrix)

    # Initiating the temporary count array
    Count_array_test = [0] * dense_matrix.shape[0]

    # Computing matrix vector product
    array_output = crs_matrix_vector_multiplication(values, column, row, array_input, Count_array_test)

    # computing the eucledian norm of the vector obtained by taking the element wise 
    # known RHS and result of Matrix vector product and checking if the difference is less than epsilon
    if(abs(vector_euclidean_norm(array_output) - vector_euclidean_norm(array_rhs)) > epsilon):
        print("Test 1 failed")
        test_failure_count += 1

    return test_failure_count

# Function for initializing of all the variables required for test_1
def init_test1_case1(size_identity, constant_array_element):
    # Create an Identity matrix of size "size_identity" using numpy module
    dense_test1_case1 = np.identity(size_identity)

    # Create an array of only "constant_array_element's" of size "size_identity"
    array_input_test1_case1 = [constant_array_element] * size_identity

    # RHS of the matrix vector product is the same as input array
    array_rhs_test1_case1 = array_input_test1_case1
    
    return (dense_test1_case1, array_input_test1_case1, array_rhs_test1_case1)

# Function for initializing of all the variables required for test_1
def init_test1_case2(size_matrix, constant_array_element):
    # Create a random sparse from built in sci-py module of "size_matrix"
    dense_test1_case2 = random(size_matrix, size_matrix, density=0.2, format='csr')

    # Create an array of only "constant_array_element's" of size "size_identity"

    array_input_test1_case2 = [constant_array_element] * size_matrix
    
    # RHS of the matrix vector product is the same as input array
    array_rhs_test1_case2 = np.dot(dense_test1_case2.todense(), np.array(array_input_test1_case2))
    array_rhs_test1_case2 = array_rhs_test1_case2.tolist()
    array_rhs_test1_case2 = array_rhs_test1_case2[0]
    return (dense_test1_case2, array_input_test1_case2, array_rhs_test1_case2)


In [31]:
import pandas as pd

if __name__ == "__main__":

     
    # Fixing the epsilon
    epsilon = 1e-10 # To account for the approximations due to floating point arithmetics

    # Initializing the test_failure_count with 0
    test_failure_count = 0
    test_failure_count_main = 0
    
    # Testing routines

    # Initializing and running test-1 and case-1 for these inputs
    # Condition-1 -----> Large Dense matrix = Identity of size 10000
    #             -----> Input vector = positive constant value(5.0) as all elements
    #             -----> Expected output = Input vector
    dense_test1_case1_1, array_input_test1_case1_1, array_rhs_test1_case1_1 = init_test1_case1(10000, 5.0)
    test_failure_count_main += test_1(dense_test1_case1_1, array_input_test1_case1_1, array_rhs_test1_case1_1, 
                            test_failure_count, epsilon)

    # Condition-2 -----> Small Dense matrix = Identity of size 1
    #             -----> Input vector = large positive constant value(10000.0) as all elements
    #             -----> Expected output = Input vector
    dense_test1_case1_2, array_input_test1_case1_2, array_rhs_test1_case1_2 = init_test1_case1(1, 10000.0)
    test_failure_count_main += test_1(dense_test1_case1_2, array_input_test1_case1_2, array_rhs_test1_case1_2, 
                            test_failure_count, epsilon)

    # Condition-3 -----> Moderate Dense matrix = Identity of size 500
    #             -----> Input vector = Negetive large constant value(-5000.0) as all elements
    #             -----> Expected output = Input vector
    dense_test1_case1_3, array_input_test1_case1_3, array_rhs_test1_case1_3 = init_test1_case1(500, -5000.0)
    test_failure_count_main += test_1(dense_test1_case1_3, array_input_test1_case1_3, array_rhs_test1_case1_3, 
                            test_failure_count, epsilon)

    # Initializing and running test-1 and case-2 for these inputs
    # Condition-1 -----> Large Dense matrix = random matrix of size 1000
    #             -----> Input vector = positive constant value(5.0) as all elements
    #             -----> Expected output = Input vector
    dense_test1_case2_1, array_input_test1_case2_1, array_rhs_test1_case2_1 = init_test1_case2(1000, 5.0)
    test_failure_count_main += test_1((np.array(dense_test1_case2_1.todense())), array_input_test1_case2_1, array_rhs_test1_case2_1, 
                            test_failure_count, epsilon)

    # Condition-2 -----> Small Dense matrix = random matrix of size 1
    #             -----> Input vector = large positive constant value(10000.0) as all elements
    #             -----> Expected output = Input vector
    dense_test1_case2_2, array_input_test1_case2_2, array_rhs_test1_case2_2 = init_test1_case2(1, 10000.0)
    test_failure_count_main += test_1((np.array(dense_test1_case2_2.todense())), array_input_test1_case2_2, array_rhs_test1_case2_2, 
                            test_failure_count, epsilon)

    # Condition-3 -----> Moderate Dense matrix = random matrix of size 50
    #             -----> Input vector = Negetive large constant value(-5000.0) as all elements
    #             -----> Expected output = Input vector
    dense_test1_case2_3, array_input_test1_case2_3, array_rhs_test1_case2_3 = init_test1_case2(50, -5000.0)
    test_failure_count_main += test_1((np.array(dense_test1_case2_3.todense())), array_input_test1_case2_3, array_rhs_test1_case2_3, 
                            test_failure_count, epsilon)

    if(test_failure_count_main == 0):
        print("All the test cases passed \n")
    else:
        print("Please check the final count of the test_failure_count_main and run all the tests independently")


    # Once we know all the test cases are passed and all the function are working as intended, we can solve the main puzzle itself

    # N = Number of Hops given as input
    N = 100

    # Using the CRS format for further computations
    NonZeroValues, ColumnIndex, RowCount = crs_storage(matrix_possible_dials)

    # Storing all the key in an array
    dial_keys_array = [keys for keys in range(10)]

    # Calling the function "Count_array_nhops" for computing the the number of possible outcomes for each key
    count_array_nhops = count_N_runs(N)
    
    # Dataframe to represent counts for each key given N hops as input
    dataframe_counts = pd.DataFrame(list(zip(dial_keys_array, count_array_nhops)), columns=['keys', 'count'])
    print("The count per key considering {} hops is : \n {}".format(N,dataframe_counts))
    print("\nThe sum of all the possibiliteis for all keys for {} hops is {}".format(N, arraySum(count_array_nhops)))


All the test cases passed 

The count per key considering 100 hops is : 
    keys         count
0     0  1.044129e+36
1     1  8.447180e+35
2     2  6.453071e+35
3     3  8.447180e+35
4     4  1.044129e+36
5     5  0.000000e+00
6     6  1.044129e+36
7     7  8.447180e+35
8     8  6.453071e+35
9     9  8.447180e+35

The sum of all the possibiliteis for all keys for 100 hops is 7.801872555471736e+36


# I would like to use the built in sci-py module to solve the same problem by storing the matrix in a sparse CRS format. At last we can compare the results with vanilla python implementation

In [32]:
# Using compressed row storage format to store the elements of the matrix as the matrix is evidently sparse.
matrix_possible_dials_crs = csr_matrix(matrix_possible_dials)

In [33]:
# Initializing a conunt vector which consists of
count_vector = np.float64(np.ones(matrix_possible_dials_crs.shape[0]))

In [34]:
# Fixing the number of hops
N_hops = 100
# Initialize the count vector for the trivial case of N_hops = 0, with vectors of 1's

def count_nhops(N):
    V = count_vector
    for i in range(N):
        V = matrix_possible_dials_crs.dot(V)
    print("The sum of all the possible outcomes for {} hops is {}".format(N_hops, np.sum(V)))

count_nhops(N_hops)

The sum of all the possible outcomes for 100 hops is 7.801872555471734e+36


# The final result matches with the built in function result.